In [1]:
import sys,os
sys.path.append("../")
import gensim
from gensim.models import KeyedVectors
from gensim.models.fasttext import FastText as FT_gensim
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from IPython.display import HTML
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy import stats
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer

## reading the change object and clustering.

In [2]:
article_name = "Truth"
change_object_dir =  "../data/change objects/"
filename =  f"{article_name}_change.h5"

change_object_file = os.path.join(change_object_dir, filename)

In [3]:
if os.path.exists(change_object_file):
    with pd.HDFStore(change_object_file, 'r') as store:
        change_object_dataframe = store.get("data")
else:
    print("file do not exist")
change_object_dataframe.shape

(31777, 6)

## Reduce the change object.

In [4]:
change_object_dataframe["ins_length"]= change_object_dataframe["ins_tokens"].apply(lambda x: len(x))
change_object_dataframe["del_length"]= change_object_dataframe["del_tokens"].apply(lambda x: len(x))
change_object_dataframe.head(5)

ins_tokens  \
from revision id to revision id timestamp           timegap          editor                                                                
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0                                       (-, -, -, -)   
                                                                                    1                                       (foregoing,)   
                                                                                    2  ([[, larrys, text, ]], ., if, you, can, do, be...   
                                                                                    3  (you, can, feel, free, to, radically, update, ...   
                                                                                    4                                                 ()   

                                                                                         del_tokens  \
from revision id to revision id timestamp           timegap          editor                           
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0            ()   
                                                                                    1            ()   
                                                                                    2            ()   
                                                                                    3            ()   
                                                                                    4  (following,)   

                                                                                                    left_neigh  \
from revision id to revision id timestamp           timegap          editor                                      
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0        slice(0, 3, None)   
                                                                                    1        slice(0, 8, None)   
                                                                                    2       slice(0, 10, None)   
                                                                                    3  slice(4212, 4243, None)   
                                                                                    4        slice(0, 3, None)   

                                                                                               right_neigh  \
from revision id to revision id timestamp           timegap          editor                                  
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0   slice(1, 32, None)   
                                                                                    1   slice(4, 35, None)   
                                                                                    2   slice(9, 40, None)   
                                                                                    3  slice(13, 44, None)   
                                                                                    4   slice(4, 35, None)   

                                                                                                                              left_token  \
from revision id to revision id timestamp           timegap          editor                                                                
286436           286907         2001-03-21 00:36:45 46 days 20:40:01 0|Larry_Sanger 0                                  ({st@rt}, <, the)   
                                                                                    1   ({st@rt}, <, the, following, is, a, portion, of)   
                                                                                    2  ({st@rt}, <, the, following, is, a, portion, o...   
                                                                                    3  (to, rest, content, with, any, sort, of, relat...   
             

In [5]:
inner_join_change_object = change_object_dataframe[(change_object_dataframe["ins_tokens"]!=()) & (change_object_dataframe["del_tokens"]!=())]
display(inner_join_change_object.shape)
inner_join_change_object.head()

(11562, 8)

ins_tokens  \
from revision id to revision id timestamp           timegap           editor                                                                    
86265            92264          2002-06-06 01:59:44 100 days 10:16:33 122               2                                               ([[,)   
                                                                                        3                                               (]],)   
                                                                                        14                                          (note, :)   
92264            165756         2002-06-12 02:59:57 6 days 01:00:13   0|217.168.172.202 0                                           (peirce,)   
353876           431091         2002-10-12 23:05:02 24 days 10:22:42  216               0   (is, the, subject, of, much, theorizing, by, p...   

                                                                                                          del_tokens  \
from revision id to revision id timestamp           timegap           editor                                           
86265            92264          2002-06-06 01:59:44 100 days 10:16:33 122               2                  (<, i, >)   
                                                                                        3               (<, /, i, >)   
                                                                                        14                      (<,)   
92264            165756         2002-06-12 02:59:57 6 days 01:00:13   0|217.168.172.202 0                  (pierce,)   
353876           431091         2002-10-12 23:05:02 24 days 10:22:42  216               0   (?, we, ’, ll, look, at)   

                                                                                                         left_neigh  \
from revision id to revision id timestamp           timegap           editor                                          
86265            92264          2002-06-06 01:59:44 100 days 10:16:33 122               2        slice(0, 27, None)   
                                                                                        3        slice(8, 39, None)   
                                                                                        14  slice(4205, 4236, None)   
92264            165756         2002-06-12 02:59:57 6 days 01:00:13   0|217.168.172.202 0   slice(2598, 2629, None)   
353876           431091         2002-10-12 23:05:02 24 days 10:22:42  216               0       slice(16, 47, None)   

                                                                                                        right_neigh  \
from revision id to revision id timestamp           timegap           editor                                          
86265            92264          2002-06-06 01:59:44 100 days 10:16:33 122               2       slice(30, 61, None)   
                                                                                        3       slice(43, 74, None)   
                                                                                        14  slice(4237, 4268, None)   
92264            165756         2002-06-12 02:59:57 6 days 01:00:13   0|217.168.172.202 0   slice(2630, 2661, None)   
353876           431091         2002-10-12 23:05:02 24 days 10:22:42  216               0       slice(53, 84, None)   

                                                                                                                                   left_token  \
from revision id to revision id timestamp           timegap           editor                                                                    
86265            92264          2002-06-06 01:59:44 100 days 10:16:33 122               2   ({st@rt}, what, is, truth, ?, we, ’, ll, look,...   
                                                                                        3   (look, at, a, number, of, answers, to, this, q...   
                          

### Removing change object with insert or delete token size more than five.

In [6]:
bykau_change_object = inner_join_change_object[~((inner_join_change_object["ins_length"] >5 ) | (inner_join_change_object["del_length"] >5) )]
bykau_change_object.shape

(6608, 8)

In [7]:
bykau_change_object.reset_index(drop=True)[["ins_tokens", "del_tokens"]].head(10)

,ins_tokens,del_tokens
0,"([[,)","(<, i, >)"
1,"(]],)","(<, /, i, >)"
2,"(note, :)","(<,)"
3,"(peirce,)","(pierce,)"
4,"(consider,)","(the,)"
5,"(associated, with, bertrand, russell, .)","(and,)"
6,"(deflationary,)","(redundancy,)"
7,"(deflationary,)","(redundancy,)"
8,"(deflationary,)","(redundancy,)"
9,"(deflationary,)","(redundancy,)"


### Removing low user support tokens

In [8]:
bykau_change_object = bykau_change_object.reset_index()
bykau_change_object = bykau_change_object.groupby("ins_tokens").filter(lambda x : x["editor"].nunique()>=2)
bykau_change_object = bykau_change_object.groupby("del_tokens").filter(lambda x : x["editor"].nunique()>=2)
bykau_change_object.shape

(3425, 14)

In [9]:
bykau_change_object.head()

,from revision id,to revision id,timestamp,timegap,editor,level_5,ins_tokens,del_tokens,left_neigh,right_neigh,left_token,right_token,ins_length,del_length
0,86265,92264,2002-06-06 01:59:44,100 days 10:16:33,122,2,"([[,)","(<, i, >)","slice(0, 27, None)","slice(30, 61, None)","({st@rt}, what, is, truth, ?, we, ’, ll, look,...","(correspondence, <, /, i, >, <, i, >, theory, ...",1,3
1,86265,92264,2002-06-06 01:59:44,100 days 10:16:33,122,3,"(]],)","(<, /, i, >)","slice(8, 39, None)","slice(43, 74, None)","(look, at, a, number, of, answers, to, this, q...","(,, and, we, can, define, it, as, follows, :, ...",1,4
3,92264,165756,2002-06-12 02:59:57,6 days 01:00:13,0|217.168.172.202,0,"(peirce,)","(pierce,)","slice(2598, 2629, None)","slice(2630, 2661, None)","(of, truth, or, of, justification, ., i, ’, ll...","(]], (, pronounced, "", purse, "", ), and, [[, w...",1,1
4,353876,431091,2002-10-12 23:05:02,24 days 10:22:42,216,2,"(consider,)","(the,)","slice(34, 65, None)","slice(66, 97, None)","(truth, ]], ., ), what, is, ', ', ', truth, ',...","(first, is, called, the, [[, correspondence, t...",1,1
12,431091,440545,2002-11-18 04:05:56,36 days 05:00:54,0|195.92.168.167,23,"(conception,)","(theory,)","slice(2152, 2183, None)","slice(2184, 2215, None)","(dispute, now, ., the, dispute, quickly, becom...","(of, truth, ,, that, differs, quite, a, bit, b...",1,1


### Clustering using jaccard

##### Defining jaccard similarity function.

In [11]:
edit_tokens = bykau_change_object["ins_tokens"] + bykau_change_object["del_tokens"]
neighbour_tokens = bykau_change_object["left_token"] + bykau_change_object["right_token"]
bykau_change_object["edit_tokens"] = edit_tokens.apply(lambda x: np.unique(x))
bykau_change_object["neighbour_tokens"] = neighbour_tokens.apply(lambda x: np.unique(x))
neighbour_vec=MultiLabelBinarizer().fit_transform(bykau_change_object["neighbour_tokens"])

In [12]:
db = DBSCAN(eps=0.5, min_samples=5, metric='jaccard').fit(neighbour_vec)
bykau_change_object["neighbour_clusters"] = db.labels_

In [64]:
group_edit_tokens = bykau_change_object.groupby("neighbour_clusters")["edit_tokens"].apply(lambda x: set(np.concatenate(x.values.ravel(), axis=0)))
context_vec = MultiLabelBinarizer().fit_transform(group_edit_tokens[1:])
group_edit_dataframe = pd.DataFrame(group_edit_tokens.reset_index()[1:])

In [65]:
db = DBSCAN(eps=0.01, min_samples=2, metric='jaccard').fit(context_vec)
group_edit_dataframe["reclustered_group"] = db.labels_

In [67]:
final_dataframe = pd.merge( bykau_change_object, group_edit_dataframe[["reclustered_group", "neighbour_clusters"]])

### Grouping and showing clusters

In [69]:
final_dataframe["left_string"] = final_dataframe["left_token"].str.join(" ")
final_dataframe["ins_string"] = final_dataframe["ins_tokens"].str.join(" ")
final_dataframe["del_string"] = final_dataframe["del_tokens"].str.join(" ")
final_dataframe["right_string"] = final_dataframe["right_token"].str.join(" ")

In [70]:
final_dataframe_grouped = final_dataframe.groupby("reclustered_group")

In [78]:
repers = final_dataframe_grouped[["left_string","del_string", "ins_string", "right_string"]].apply(lambda x: x.style.render())

In [76]:
# all_html = " ".join(repers_4_full)
# file_name = article_name + "_4_full_"+str(NO_OF_CLUSTERS) + "_clusters.html"
# file_path = os.path.join("./visualisation", file_name)
# with open(file_path, 'wb') as f:
#     f.write(all_html.encode())

In [82]:
@interact( clusters_html=fixed(repers), group=range(final_dataframe_grouped.ngroups))
def display_clusters(clusters_html, group):
     return display(HTML(clusters_html.iloc[group]))


interactive(children=(Dropdown(description='group', options=(0, 1, 2, 3, 4, 5, 6, 7), value=0), Output()), _do…